In [31]:
import scipy.io
import numpy as np
import pandas as pd

from label_dict import label_dict

mat = scipy.io.loadmat(r'..\data\ninapro_DB1\S1_A1_E2.mat')

In [32]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'emg', 'stimulus', 'glove', 'subject', 'exercise', 'repetition', 'restimulus', 'rerepetition'])

In [33]:
rep = mat["rerepetition"].copy()
emg = mat["emg"].copy()
lab = mat["restimulus"].copy() # label

new_lab = np.array([[label_dict[2][lab[i][0]]] for i in range(lab.shape[0])])

In [34]:
for i in range (465):
    if new_lab[i] != 0:
        print (f"Rep nr: {rep[i][0]}, Class: {new_lab[i][0]}, Frame nr: {i}, EMG: {emg[i]}")

Rep nr: 1, Class: 13, Frame nr: 454, EMG: [0.0488 0.0098 0.0024 0.0024 0.0049 0.0024 0.4077 0.166  0.0024 0.0659]
Rep nr: 1, Class: 13, Frame nr: 455, EMG: [0.0708 0.0146 0.0024 0.0024 0.0024 0.0024 0.4858 0.2002 0.0024 0.0781]
Rep nr: 1, Class: 13, Frame nr: 456, EMG: [0.083  0.0122 0.0024 0.0024 0.0024 0.0024 0.542  0.2148 0.0024 0.0879]
Rep nr: 1, Class: 13, Frame nr: 457, EMG: [0.0854 0.0024 0.0024 0.0024 0.0024 0.0024 0.5859 0.2148 0.0024 0.0879]
Rep nr: 1, Class: 13, Frame nr: 458, EMG: [0.083  0.0024 0.0024 0.0073 0.0024 0.0024 0.5957 0.1978 0.0024 0.0977]
Rep nr: 1, Class: 13, Frame nr: 459, EMG: [0.0854 0.0171 0.0171 0.022  0.0024 0.0024 0.6396 0.2002 0.0586 0.1172]
Rep nr: 1, Class: 13, Frame nr: 460, EMG: [0.0903 0.0757 0.0854 0.0659 0.0024 0.0024 0.7007 0.2124 0.1636 0.1392]
Rep nr: 1, Class: 13, Frame nr: 461, EMG: [0.1196 0.1343 0.1343 0.083  0.0024 0.0024 0.7983 0.2661 0.2124 0.1929]
Rep nr: 1, Class: 13, Frame nr: 462, EMG: [0.144  0.166  0.1514 0.083  0.0024 0.0049 0.8

In [5]:
df = pd.DataFrame(np.concatenate((rep, emg, new_lab), axis=1))
df.drop(df[df[11] == 0.0].index, inplace=True)

df

,0,1,2,3,4,5,6,7,8,9,10,11
454,1.0,0.0488,0.0098,0.0024,0.0024,0.0049,0.0024,0.4077,0.1660,0.0024,0.0659,13.0
455,1.0,0.0708,0.0146,0.0024,0.0024,0.0024,0.0024,0.4858,0.2002,0.0024,0.0781,13.0
456,1.0,0.0830,0.0122,0.0024,0.0024,0.0024,0.0024,0.5420,0.2148,0.0024,0.0879,13.0
457,1.0,0.0854,0.0024,0.0024,0.0024,0.0024,0.0024,0.5859,0.2148,0.0024,0.0879,13.0
458,1.0,0.0830,0.0024,0.0024,0.0073,0.0024,0.0024,0.5957,0.1978,0.0024,0.0977,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
142643,10.0,0.1074,0.2051,0.0049,0.0024,0.0024,0.0024,0.1733,0.3076,0.1660,0.2124,29.0
142644,10.0,0.1001,0.1685,0.0024,0.0024,0.0024,0.0024,0.1514,0.2832,0.1953,0.2002,29.0
142645,10.0,0.0879,0.1343,0.0024,0.0024,0.0024,0.0024,0.1245,0.2563,0.1929,0.1904,29.0
142646,10.0,0.0854,0.1050,0.0024,0.0024,0.0024,0.0024,0.1123,0.2563,0.1733,0.1904,29.0


In [18]:
def wavelength_form(x):
    return x.diff().abs().sum()

# Average signal's energy
def av_signal_energy(x):
    return np.sum(x**2) / len(x)


In [19]:
summed_data = df.groupby([11, 0]).sum()
mean_data = df.groupby([11, 0]).mean()
waveform_data = df.groupby([11, 0]).agg(wavelength_form)
av_energy_data = df.groupby([11, 0]).agg(av_signal_energy)

final_database = pd.concat([summed_data, mean_data, waveform_data, av_energy_data], axis=1)
final_database.to_csv('output_cols.csv', index=False) 

final_database

1         2         3        4        5        6         7    
11   0                                                                         
13.0 1.0   316.9895  212.8556   94.5064  38.5705  17.9255  15.9843  242.2364  \
     2.0   310.7591  266.2786  114.9121  36.8615  17.3998  10.0807  141.7552   
     3.0   221.0061  165.1410   33.9902   9.7526   4.2481   3.2673   93.8923   
     4.0   278.1416  164.2322   38.9760  12.5851   6.2091   5.6305   79.2920   
     5.0    78.2733   49.3259   28.6610   7.5125   3.0956   1.2440   44.9809   
...             ...       ...       ...      ...      ...      ...       ...   
29.0 6.0   113.7465  213.8290   61.0274  33.2327   6.6300   6.9045  102.0110   
     7.0   126.9875  341.6264   67.3662  31.6512   5.7604   6.7379  154.0663   
     8.0   100.1199  202.2896   50.4011  27.3082   5.0788   5.0442  106.5456   
     9.0   125.6257  168.4533   60.8338  28.8812   5.9735   6.8808  157.6768   
     10.0   99.2591  149.0482   46.7482  26.3108   4.7738   4.4959  105.0858   

                 8         9         10  ...        1         2         3    
11   0                                   ...                                 
13.0 1.0   257.0242  135.5319  127.8049  ...  1.630853  0.765143  0.164917  \
     2.0   475.6398  151.7044  125.4094  ...  1.445168  1.161231  0.247754   
     3.0   168.8596   83.4142   42.7523  ...  1.330081  0.655250  0.035474   
     4.0   137.2554   84.4953   35.5139  ...  1.667531  0.555937  0.030951   
     5.0   130.3564   27.5220   17.8404  ...  0.098363  0.047515  0.022262   
...             ...       ...       ...  ...       ...       ...       ...   
29.0 6.0   369.5656   58.4447   43.9395  ...  0.344947  1.327033  0.061969   
     7.0   414.8318   74.6647   44.1192  ...  0.191438  1.649844  0.046103   
     8.0   340.8010   49.6915   41.1668  ...  0.184779  0.718754  0.051636   
     9.0   371.3099   72.4213   45.3009  ...  0.236300  0.476584  0.049351   
     10.0  300.7229   45.7287   41.6364  ...  0.163446  0.432294  0.055406   

                 4         5         6         7         8         9    
11   0                                                                  
13.0 1.0   0.029408  0.006899  0.004707  0.765320  1.224910  0.252235  \
     2.0   0.030322  0.007202  0.002051  0.243574  3.589320  0.343306   
     3.0   0.003838  0.000925  0.000491  0.230739  0.487528  0.175169   
     4.0   0.003834  0.001003  0.000997  0.100622  0.244833  0.150007   
     5.0   0.002410  0.000529  0.000044  0.039305  0.306147  0.017432   
...             ...       ...       ...       ...       ...       ...   
29.0 6.0   0.019028  0.002192  0.002501  0.354186  3.777035  0.082927   
     7.0   0.016530  0.000842  0.001118  0.376147  2.975245  0.068072   
     8.0   0.022089  0.000732  0.000811  0.187328  2.484463  0.034912   
     9.0   0.016756  0.001385  0.001749  0.517588  2.806209  0.088871   
     10.0  0.022112  0.000838  0.000755  0.158721  1.935909  0.034538   

                 10  
11   0               
13.0 1.0   0.206170  
     2.0   0.286563  
     3.0   0.043009  
     4.0   0.021655  
     5.0   0.006161  
...             ...  
29.0 6.0   0.038245  
     7.0   0.013850  
     8.0   0.011769  
     9.0   0.015828  
     10.0  0.012384  

[170 rows x 40 columns]